# Sensors measures

## 0. Clean Data (Open Refine)

## 1. Read Files

In [6]:
import pandas as pd
import os

csv_files = list()
json_files = list()

rel_path="../data/raw/"
for root, dirs, files in os.walk(rel_path):
    for filename in files:
        split_filename = filename.split('-')
        if len(split_filename) == 3 and filename.split('.')[1] == 'csv':
            csv_files.append({"data":pd.read_csv(rel_path + filename, parse_dates =["time"], index_col ="time"),"type":split_filename[0],"sensor":split_filename[1]})
        if len(split_filename) == 3 and filename.split('.')[1] == 'json':
            json_files.append({"data":pd.read_json(rel_path + filename,lines=True,convert_dates=['time']),"type":split_filename[0],"sensor":split_filename[1]})
print(csv_files[:1])
print(json_files[:1])

[{'data':                             sensor  value
time                                     
2017-12-19 14:07:18+00:00  H-DHT22   15.7
2017-12-19 14:07:25+00:00  H-DHT22   15.7
2017-12-19 14:07:32+00:00  H-DHT22   15.7
2017-12-19 14:07:38+00:00  H-DHT22   15.7
2017-12-19 14:07:45+00:00  H-DHT22   15.7
...                            ...    ...
2018-03-13 11:41:54+00:00  H-DHT22    1.0
2018-03-13 11:41:58+00:00  H-DHT22    1.0
2018-03-13 11:42:02+00:00  H-DHT22    1.0
2018-03-13 11:42:06+00:00  H-DHT22    1.0
2018-03-13 11:42:10+00:00  H-DHT22    1.0

[1440160 rows x 2 columns], 'type': 'H', 'sensor': 'DHT22'}]
[{'data':     sensor    value                      time
0  P-DM280  1040.00 2017-12-22 10:51:32+00:00
1  P-DM280  1040.00 2017-12-22 10:51:35+00:00
2  P-DM280  1039.97 2017-12-22 10:51:38+00:00
3  P-DM280  1039.95 2017-12-22 10:51:41+00:00, 'type': 'P', 'sensor': 'DM280'}]


## 2. Clean Data
### 2.1. Rename columns with representable values

In [7]:
for df in (csv_files + json_files):
    if df['type']=='H':
        df['data'].rename(columns={"value":"humidity"}, inplace=True)
    if df['type'] == 'T':
        df['data'].rename(columns={"value":"temperature"}, inplace=True)
    if df['type'] == 'P':
        df['data'].rename(columns={"value":"hPa"}, inplace=True)
    df['data'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1440160 entries, 2017-12-19 14:07:18+00:00 to 2018-03-13 11:42:10+00:00
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   sensor    1440160 non-null  object 
 1   humidity  1440160 non-null  float64
dtypes: float64(1), object(1)
memory usage: 33.0+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1420308 entries, 2017-12-22 11:22:11+00:00 to 2018-03-13 11:42:09+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   sensor       1420308 non-null  object
 1   temperature  1420308 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 32.5+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1440150 entries, 2017-12-19 14:07:18+00:00 to 2018-03-13 11:42:10+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 

### 2.2. Convert time field as index on json files

In [8]:
for df in json_files:
    df['data'] = df['data'].set_index('time')
    df['data'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2017-12-22 10:51:32+00:00 to 2017-12-22 10:51:41+00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sensor  4 non-null      object 
 1   hPa     4 non-null      float64
dtypes: float64(1), object(1)
memory usage: 96.0+ bytes
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1428383 entries, 2017-12-20 12:17:05+00:00 to 2018-03-13 11:42:11+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   sensor       1428383 non-null  object 
 1   temperature  1428383 non-null  float64
dtypes: float64(1), object(1)
memory usage: 32.7+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2017-12-22 10:51:31+00:00 to 2017-12-22 10:51:41+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sensor       4 non-

### 2.3. Drop rows (incomplete or duplicated)

In [9]:
for df in (json_files + csv_files):
    df['data'].drop_duplicates(inplace=True)
    df['data'].dropna(inplace=True)
    df['data'].sort_values(by='time',inplace=True)

### 2.3. Make the mean by day

In [17]:
humidity_sensors = list()
temperature_sensors = list()
preasure_sensors = list()
for df in (csv_files+json_files):
    df['data'] = df['data'].resample('D').mean()
    df['data']['sensor'] = df['sensor']
    df['data'].dropna(inplace=True)
    if df['type']=='H':
        humidity_sensors.append(df['data'])
    if df['type'] == 'T':
        temperature_sensors.append(df['data'])
    if df['type'] == 'P':
        preasure_sensors.append(df['data'])

[                            humidity sensor
time                                       
2017-12-19 00:00:00+00:00   9.004444  DHT22
2017-12-20 00:00:00+00:00  11.500000  DHT22,                            temperature sensor
time                                         
2017-12-22 00:00:00+00:00         25.0  DHT11
2017-12-23 00:00:00+00:00         19.5  DHT11
2017-12-24 00:00:00+00:00         18.0  DHT11
2017-12-25 00:00:00+00:00         17.0  DHT11
2017-12-26 00:00:00+00:00         16.0  DHT11
2017-12-29 00:00:00+00:00         15.0  DHT11,                            temperature sensor
time                                         
2017-12-19 00:00:00+00:00    25.129787  DHT22
2017-12-20 00:00:00+00:00    25.077273  DHT22
2017-12-22 00:00:00+00:00    29.130769  DHT22
2017-12-23 00:00:00+00:00    21.300000  DHT22
2017-12-24 00:00:00+00:00    20.350000  DHT22
2017-12-25 00:00:00+00:00    19.250000  DHT22
2017-12-26 00:00:00+00:00    18.300000  DHT22
2017-12-27 00:00:00+00:00    17.600000 